In [1]:
import cudf
import dask_cudf

In [2]:
import dask_cudf

In [3]:
from dask_cuml.linear_regression import LinearRegression

In order to run this notebook, you will first need to run a dask scheduler and number of dask workers:
- Run a dask scheduler with:  ```dask-scheduler --scheduler-file=cluster.json```
- Run N dask workers with:  ```mpirun -np N dask-mpi --no-nanny --nthreads 10 --no-scheduler --scheduler-file cluster.json```

In [4]:
from dask_cuda import LocalCUDACluster
cluster = LocalCUDACluster(threads_per_worker = 10)


In [5]:
from dask.distributed import Client
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:43842 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 20 Memory: 50.39 GB


In [6]:
import numba.cuda

devs = [i.id for i in numba.cuda.cudadrv.devices.gpus]
workers = list(client.has_what().keys())
worker_devs = workers[0:min(len(devs), len(workers))]

In [7]:
def set_visible(i, n):
    import os, numba.cuda
    all_devices = list(range(n))
    vd = ",".join(map(str, all_devices[i:] + all_devices[:i]))
    print(str(vd))
    numba.cuda.close()
    numba.cuda.select_device(i)
    print("Selecting Device : "  + str(i))
    os.environ["CUDA_VISIBLE_DEVICES"] = vd

dev_assigned = [client.submit(set_visible, dev, len(devs), workers = [worker]) for dev, worker in zip(devs, worker_devs)]

In [8]:
import pandas as pd

X = cudf.DataFrame([('a', [0, 1, 2, 3, 4])])
y = cudf.Series([0, 1, 2, 3, 4])


In [9]:
X_df = dask_cudf.from_cudf(X, chunksize=1).persist()
y_df = dask_cudf.from_cudf(y, chunksize=1).persist()

In [10]:
import numba.cuda
import cuml
def print_device(arr):
    print(str(numba.cuda.get_current_device()))
    print(str(cuml.device_of_ptr(arr.as_gpu_matrix(order="F"))))
    
[client.submit(print_device, part) for part in X_df.to_delayed()]

[<Future: status: pending, key: print_device-dfd22f02c29d8058874c59d72b773408>,
 <Future: status: pending, key: print_device-743d9fd427c0be1c37ff81818eea7437>,
 <Future: status: pending, key: print_device-4cacf6083130280f857a1c59b57cbfef>,
 <Future: status: pending, key: print_device-3cb46d03479458cff2d1845570acbb95>]

In [11]:
client.who_has()

{"('from_cudf-855267eec9af4ab7be8d7e2a0d7c0d73', 0)": (),
 "('from_cudf-855267eec9af4ab7be8d7e2a0d7c0d73', 1)": (),
 "('from_cudf-855267eec9af4ab7be8d7e2a0d7c0d73', 2)": (),
 "('from_cudf-855267eec9af4ab7be8d7e2a0d7c0d73', 3)": (),
 "('from_cudf-b454da2931be4697a20871a46a3343fb', 0)": (),
 "('from_cudf-b454da2931be4697a20871a46a3343fb', 1)": (),
 "('from_cudf-b454da2931be4697a20871a46a3343fb', 2)": (),
 "('from_cudf-b454da2931be4697a20871a46a3343fb', 3)": (),
 'print_device-3cb46d03479458cff2d1845570acbb95': (),
 'print_device-4cacf6083130280f857a1c59b57cbfef': (),
 'print_device-743d9fd427c0be1c37ff81818eea7437': (),
 'print_device-dfd22f02c29d8058874c59d72b773408': (),
 'set_visible-20a8edbe2b929812aef46b88c316af1b': ('tcp://127.0.0.1:40584',),
 'set_visible-6ad2ab08bd232ecacdd3435ec970aa4d': ('tcp://127.0.0.1:41423',)}

Set each worker to host dfs on a different device. 

__Note__: You can ignore this if you started your workers with "CUDA_VISIBLE_DEVICE" already

In [12]:
client.who_has()

{"('from_cudf-855267eec9af4ab7be8d7e2a0d7c0d73', 0)": (),
 "('from_cudf-855267eec9af4ab7be8d7e2a0d7c0d73', 1)": (),
 "('from_cudf-855267eec9af4ab7be8d7e2a0d7c0d73', 2)": (),
 "('from_cudf-855267eec9af4ab7be8d7e2a0d7c0d73', 3)": (),
 "('from_cudf-b454da2931be4697a20871a46a3343fb', 0)": (),
 "('from_cudf-b454da2931be4697a20871a46a3343fb', 1)": (),
 "('from_cudf-b454da2931be4697a20871a46a3343fb', 2)": (),
 "('from_cudf-b454da2931be4697a20871a46a3343fb', 3)": (),
 'print_device-3cb46d03479458cff2d1845570acbb95': (),
 'print_device-4cacf6083130280f857a1c59b57cbfef': (),
 'print_device-743d9fd427c0be1c37ff81818eea7437': (),
 'print_device-dfd22f02c29d8058874c59d72b773408': (),
 'set_visible-20a8edbe2b929812aef46b88c316af1b': ('tcp://127.0.0.1:40584',),
 'set_visible-6ad2ab08bd232ecacdd3435ec970aa4d': ('tcp://127.0.0.1:41423',)}

In [13]:
lr = LinearRegression()

In [14]:
res = lr.fit(X_df, y_df)

input_devarrays: [(('127.0.0.1', 40584), <Future: status: finished, type: tuple, key: inputs_to_device_arrays-e4d64533b0f483b000bbfaa14466d79f>), (('127.0.0.1', 41423), <Future: status: finished, type: tuple, key: inputs_to_device_arrays-0324cee2d6c59a56568dac4f67f5aeab>)]
exec_node: ('127.0.0.1', 40584)
ipc_handles: [<Future: status: pending, key: get_input_ipc_handles-fc2823cc494b24b799416b84e68e2b1a>]
raw_arrays: [<Future: status: finished, type: tuple, key: inputs_to_device_arrays-e4d64533b0f483b000bbfaa14466d79f>]
COEFS: (('127.0.0.1', 40584), <Future: status: pending, key: extract_part-2b67a2e7fc48a8954f70542c93405cb1>)
INTER: <Future: status: pending, key: extract_part-7b6f5bb8c5ec2eaa2b1fdca2b9d3312d>
RES: <Future: status: pending, key: extract_part-7b6f5bb8c5ec2eaa2b1fdca2b9d3312d>


res

In [15]:
lr.intercept_.result()

5

In [16]:
client.who_has()

{"('from_cudf-855267eec9af4ab7be8d7e2a0d7c0d73', 0)": ('tcp://127.0.0.1:40584',),
 "('from_cudf-855267eec9af4ab7be8d7e2a0d7c0d73', 1)": ('tcp://127.0.0.1:41423',),
 "('from_cudf-855267eec9af4ab7be8d7e2a0d7c0d73', 2)": ('tcp://127.0.0.1:40584',),
 "('from_cudf-855267eec9af4ab7be8d7e2a0d7c0d73', 3)": ('tcp://127.0.0.1:41423',),
 "('from_cudf-b454da2931be4697a20871a46a3343fb', 0)": ('tcp://127.0.0.1:40584',),
 "('from_cudf-b454da2931be4697a20871a46a3343fb', 1)": ('tcp://127.0.0.1:41423',),
 "('from_cudf-b454da2931be4697a20871a46a3343fb', 2)": ('tcp://127.0.0.1:40584',),
 "('from_cudf-b454da2931be4697a20871a46a3343fb', 3)": ('tcp://127.0.0.1:41423',),
 '_fit_on_worker-b67bb9fdb3777f38016a9fb3048aa6b4': (),
 'extract_part-2b67a2e7fc48a8954f70542c93405cb1': ('tcp://127.0.0.1:40584',),
 'extract_part-7b6f5bb8c5ec2eaa2b1fdca2b9d3312d': ('tcp://127.0.0.1:40584',),
 'get_input_ipc_handles-fc2823cc494b24b799416b84e68e2b1a': (),
 'get_result-b6801ed2a920acc348406185e0f7c09f': (),
 'inputs_to_devic

In [17]:
g = lr.predict(X_df)

WORKER PARTS: [(('127.0.0.1', 41423), <Future: status: finished, type: DataFrame, key: ('from_cudf-b454da2931be4697a20871a46a3343fb', 3)>), (('127.0.0.1', 40584), <Future: status: finished, type: DataFrame, key: ('from_cudf-b454da2931be4697a20871a46a3343fb', 2)>), (('127.0.0.1', 41423), <Future: status: finished, type: DataFrame, key: ('from_cudf-b454da2931be4697a20871a46a3343fb', 1)>), (('127.0.0.1', 40584), <Future: status: finished, type: DataFrame, key: ('from_cudf-b454da2931be4697a20871a46a3343fb', 0)>)]
ON WORKER: 2
NOT ON WORKER: 2
IPCHANDLES = [<Future: status: pending, key: get_ipc_handles-e70e8a3ca1ecb66ee3e131ff57764f01>, <Future: status: pending, key: get_ipc_handles-44f597fb7085444c4accf99c0695d675>]
RAW_ARRAYS=[<Future: status: pending, key: as_gpu_matrix-b36bc8e94ae0666b376df0f08a9870f8>, <Future: status: pending, key: as_gpu_matrix-9a7c936ad4fff63b7121010655fcf141>]
f=<Future: status: finished, type: tuple, key: _predict_on_worker-319815a5a074541dffe2d1dd0515887b>


In [27]:
print(str(g.result()))

      
0    1
1    2
2    3
3    4
4    5
